<a href="https://colab.research.google.com/github/willdone1337/NLP_poetry_generator/blob/master/markovify_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Loading the model from pickle file
import numpy as np
from flask import Flask, request, make_response
import json
import pickle
from tensorflow.keras.models import load_model
import re
import os.path
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
import pandas as pd
from tensorflow.keras import regularizers
import numpy as np
from keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import load_model
from keras.layers import LSTM, Dense, Dropout, Flatten
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.optimizers import RMSprop
from keras.utils import np_utils

import os
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
!pip install markovify
import markovify
import random

In [5]:
data = open('/content/gdrive/MyDrive/models/nizami_books.txt',encoding='utf-8').read().lower().split('\n')
data=list(data)
model=markovify.NewlineText(data)
import numpy as np

for i in range(5):
    print()
    for i in range(random.randrange(1, 4)):
        print(model.make_short_sentence(30))


bil ki, bu dünyada biz,
qərəzli bir sövda daşı,
yadında bir neçə ağac,

təklikdə ağlardı o gözəl pəri.

None
bir quş idi o gün.
None

None

gəl bu yerdə bir dəm.
özün də o sinəndədir.
elə ki, başa çatdı birtəhər,


In [7]:
chars = (set(data))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

vocab_size = len(chars)
print('Vocabulary size: {}'.format(vocab_size))

Vocabulary size: 14529


In [8]:
X = [] 
Y = [] 

length = len(data)
seq_length = 100

for i in range(0, length - seq_length, 1):
    sequence = data[i:i + seq_length]
    label = data[i + seq_length]
    X.append([char_indices[char] for char in sequence])
    Y.append(char_indices[label])

print('NUum of seq: {}'.format(len(X)))

NUum of seq: 15506


In [9]:
X_new = np.reshape(X, (len(X), seq_length, 1))
X_new = X_new/float(len(chars))
Y_new = np_utils.to_categorical(Y)

X_new.shape, Y_new.shape

((15506, 100, 1), (15506, 14529))

In [10]:
model = Sequential()
model.add(LSTM(150, input_shape = (X_new.shape[1], X_new.shape[2]), return_sequences = True))

model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(Y_new.shape[1], activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

model.fit(X_new, Y_new, epochs = 1, verbose = 1)

485/485 [==============================] - 61s 59ms/step - loss: 9.5233


In [11]:
start = np.random.randint(0, len(X)-1)
string_mapped = list(X[start])
full_string = [indices_char[value] for value in string_mapped]

# Generate text
for i in range(30):
    x = np.reshape(string_mapped, (1, len(string_mapped), 1))
    x = x / float(len(chars))
    
    pred_index = np.argmax(model.predict(x, verbose = 0))
    seq = [indices_char[value] for value in string_mapped]
    full_string.append(indices_char[pred_index])
    
    string_mapped.append(pred_index)
    string_mapped = string_mapped[1:len(string_mapped)]
    
# Combine text
newtext = 'əgər '
for char in full_string:
    newtext = newtext + char

print(newtext)

əgər sənin məhrəmindir bu gözəl can da.səndən söz yaratmaq gözlənsin gərək,məndən dua etmək, bəxtdən də kömək",danışıb qurtardı o ciyərparam,onun xatirinə söz aldı ilham.gövhər axtarmağa başladım yenə,kimyalar açaraq girdim mədənə.təbim də ən qısa bir yol axtardı,yol uzunluğundan qorxum da vardı.nə bundan qısaca yol var hələlik,nə də bundan qıvraq, iti bir minik.bəhrin suyu yüngül, özü axardır,canlı, nəfəs alan balığı vardırçox bu şirinlikdə söz deyən olar,sanma ki, onlarda bu təzəlik var.hələ üzücülər qəlb dənizindənbelə saf bir inci tapmamış, bilsən.hər beyti, sapdakı incilər kimi,parlayır eybsiz bir hünər kimi.mən bu inciləri gəzəndə bir-bir,ayağım tük qədər sürüşməmişdir.qəlb cavab- verirdi mənə ürəkdən,bulaq su verirdi yer qazdıqca mən.əqildən aldığım bu gəlir, dövlətyalnız əsərimə vururdu zinət.dörd min beytdən də çoxdur bu dastan,dörd aydan az vaxta yazmışam, inan!əgər başqa işlər olsaydı haram,bu, on dörd gecəyə olardı tamam.bu azad gəlini görən şad olsun,ona abad deyən, qoy ab

In [12]:
import keras.utils as ku
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding

# Lowercase all text


# Create Tokenizer object to convert words to sequences of integers
tokenizer = Tokenizer(num_words = None, filters = '#$%&(),*+-<=>@[\\]^_`{|}~\t\n', lower = False)

# Train tokenizer to the texts
tokenizer.fit_on_texts(data)
total_words = len(tokenizer.word_index) + 1

# Convert list of strings into flat dataset of sequences of tokens
sequences = []
for line in data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        sequences.append(n_gram_sequence)

# Pad sequences to ensure equal lengths
max_seq_len = max([len(x) for x in sequences])
sequences = np.array(pad_sequences(sequences, maxlen = max_seq_len, padding = 'pre'))

# Create n-grams sequence predictors and labels
predictors, label = sequences[:, :-1], sequences[:, -1]
label = ku.to_categorical(label, num_classes = total_words)
input_len = max_seq_len - 1
################
"""model = Sequential()
model.add(Embedding(total_words, 50, input_length = input_len))
model.add(LSTM(150))
model.add(Dropout(0.1))
model.add(Dense(total_words, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')"""
###############################################################################################


model=load_model('/content/gdrive/MyDrive/models/nizamimodel_06loss.h5')
model.summary()
################# 0.6 categorical_Crossentropy loss with 100 epochs  ##########################

#
#model.fit(predictors, label, epochs = 100, verbose = 1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 9, 50)             1178450   
_________________________________________________________________
lstm_1 (LSTM)                (None, 150)               120600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 150)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 23569)             3558919   
Total params: 4,857,969
Trainable params: 4,857,969
Non-trainable params: 0
_________________________________________________________________


In [13]:
def generate_line(text, next_words, max_seq_len, model):
    for j in range(next_words):
        token_list = tokenizer.texts_to_sequences([text])[0]
        token_list = pad_sequences([token_list], maxlen = max_seq_len - 1, padding = 'pre')
        predicted = model.predict_classes(token_list, verbose = 0)
        
        output_word = ''
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        text += ' ' + output_word
    return text
generate_line('səma',30,max_seq_len,model)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


'səma başının tacı yer üzü taxtın oldu. oldu. indi çöldə gecənin dar olum mən meydandı tapmasa sifətlər nədir? xeyri əhd keçən əlimdə mən. dürr heç var? heç zaman. gecətək qurda ölkəyə'

In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
data = open('/content/gdrive/MyDrive/models/nizami_books.txt',encoding='utf-8').read().lower().split('\n')

tokenizer=Tokenizer(num_words=3000)
tokenizer.fit_on_texts(data)
total_words=len(tokenizer.word_index)
input_sequences=[]
for line in data:
    token_list=tokenizer.texts_to_sequences([line])[0]
    for x in range(len(token_list)):
        n_gram_seq=token_list[:x+1]
        input_sequences.append(n_gram_seq)

max_lenght=max([len(x) for x in input_sequences])
padded_sequences=np.array(pad_sequences(input_sequences,maxlen=max_lenght,padding='pre'))
x,y=padded_sequences[:,:-1],padded_sequences[:,-1]
y=tf.keras.utils.to_categorical(y,num_classes=total_words)
model = load_model('/content/gdrive/MyDrive/colabv2/trynizami_47percent.h5')

def poem(x):
    next_words=100
    seed_text = ''
    for _ in range(1, next_words + 1):
        token_list = tokenizer.texts_to_sequences([seed_text + x])[0]
        token_list = pad_sequences([token_list], maxlen=max_lenght - 1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
        output_word = ''
        if _ % 4 == 0:
            seed_text = seed_text + ','
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        # if _%4==0:
        #   output_word+='\n'
        seed_text += ' ' + output_word

    split_regex = re.compile(r'[,]')
    sentences = [t.strip() for t in split_regex.split(seed_text)]
    poem = []
    for s in sentences:
        poem.append(s)
    alfa=''
    for x in poem:
        alfa+=x
    return alfa

print(poem('leyli'))

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


gül bir kimiaz şahıdır yoluna açmışdıoda bilsən inan inanolardı də dünyada indisən görək olubdur ondaşəfaət kişi göndər verənyandırar də bil ayaqüstə qərar nəfəs açıbdeyir qanad bu üstünəbaşı atdı onu onumən mən daha əzəldənlal deyil də abadsağam oldu mən ondannədir sənin qoca zəmanəçapdı hanı zənciri tökərəzəldən sona işi sənindərhal üzü çək şadsən də yaxşıdır bağışlaözünə onu birdən eyçatar qısa niyə mənbir deyil kəs olmuşdaha pis var vardıvardır əsər vardır yetirbudur onu bu boşqoca ağlardı sıxdı gərəkatdı
